In [1]:
# import neccessary library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer 
import os 
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
# Extracting Data from a Single Excel File Containing Multiple Stock Index Information
def data_cleaning(original_file_name, saved_file_name):
    index_data_dir = os.path.join(os.getcwd(), 'data', 'feature_data')
    original_file = pd.read_excel(os.path.join(index_data_dir, original_file_name))

    # [RUT:102434 DJX: 102456 SPX: 108105]
    filtered_data = original_file[original_file['Security ID'] == 108105]
    filtered_data['Date'] = filtered_data['Date'].dt.strftime('%-d/%-m/%Y')

    # Count the number of rows in the filtered DataFrame
    # Define the path for the new Excel file
    save_data_dir = os.path.join(index_data_dir, 'unfinish data')
    output_excel_path = os.path.join(save_data_dir, saved_file_name)

    # Write the filtered data to a new Excel file
    filtered_data.to_excel(output_excel_path, index=False)


In [3]:
# data_cleaning('OptionMetrics - Index Dividend Yield.xlsx', 'OptionMetrics - SPX  Index Dividend Yield.xlsx')



In [4]:
# Avoid data leakage to test data
# Impute missing data in train and test datasets using KNNImputer.
def impute_data(train, test):
    imputer = KNNImputer(n_neighbors=5)

    # Fit the imputer on the training data
    imputer.fit(train)

    # Transform both training and testing data
    train_imputed = imputer.transform(train)
    test_imputed = imputer.transform(test)

    return train_imputed, test_imputed

# get the feature data from data folder
def load_feature_data(start_date, test_start_dates, end_date, rebal_period):
    # Load index data, impute missing values and return train and test datasets
    index_data_dir = os.path.join(os.getcwd(), 'data', 'feature_data')
    result_train_df = pd.DataFrame()
    result_test_df = pd.DataFrame()
    
    index_data_name_file = {
        'CBOE DJIA Volatility Index':'CBOE DJIA Volatility Index - Close.xlsx',
        'CBOE NASDAQ Volatility Index': 'CBOE NASDAQ Volatility Index - Close.xlsx',
        'CBOE S&P100 Volatility Index': 'CBOE S&P100 Volatility Index - Close.xlsx',
        'CBOE S&P500 Volatility Index': 'CBOE S&P500 Volatility Index - Close.xlsx',
        'OptionMetrics - DJI Historical Volatility': 'OptionMetrics - DJI Historical Volatility.xlsx',
        'OptionMetrics - Russell 2000 Historical Volatility': 'OptionMetrics - Russell 2000 Historical Volatility.xlsx',
        'OptionMetrics - SPX Historical Volatility': 'OptionMetrics - SPX Historical Volatility.xlsx',
        'OptionMetrics - DJX  Index Dividend Yield': 'OptionMetrics - DJX  Index Dividend Yield.xlsx',
        'OptionMetrics - Russell 2000  Index Dividend Yield': 'OptionMetrics - Russell 2000  Index Dividend Yield.xlsx',
        'OptionMetrics - SPX  Index Dividend Yield': 'OptionMetrics - SPX  Index Dividend Yield.xlsx'
    }
    
    for ind_name in index_data_name_file.keys():
        input = pd.read_excel(os.path.join(index_data_dir, index_data_name_file[ind_name]), index_col='Date')
        # Ensure that the index is a DatetimeIndex
        input.index = pd.to_datetime(input.index)
        input = input.sort_index(ascending=False)
        
        # Calculate monthly percentage changes to enhance the stationarity of the data
        # Fill the first with 0
        index_df = input.iloc[:, 0].resample(rebal_period).first().pct_change().fillna(0)
        
         # Split the data into train and test based on the provided dates
        train_data = index_df.loc[start_date:test_start_dates]
        test_data = index_df.loc[test_start_dates:end_date]
        
        # Handle infinite values
        inf_indices_train = np.where(np.isinf(train_data))[0]
        inf_indices_test = np.where(np.isinf(test_data))[0]
        
        train_data[inf_indices_train] = np.nan
        test_data[inf_indices_test] = np.nan
        
        train_imputed, test_imputed = impute_data(train_data.values.reshape(-1, 1), test_data.values.reshape(-1, 1))
        
        # Convert back to Series
        train_imputed_series = pd.Series(train_imputed.flatten(), index=train_data.index)
        test_imputed_series = pd.Series(test_imputed.flatten(), index=test_data.index)
        
        # Rename the series for clarity
        train_imputed_series.rename(ind_name, inplace=True)
        test_imputed_series.rename(ind_name, inplace=True)

        result_train_df = pd.concat([result_train_df, train_imputed_series], axis=1)
        result_test_df = pd.concat([result_test_df, test_imputed_series], axis=1)
        
    
    result_train_df.index = pd.to_datetime(result_train_df.index)
    result_test_df.index = pd.to_datetime(result_test_df.index)

    return result_train_df, result_test_df

In [5]:
# Define general parameters
train_start_date = '2007-01-01'
test_start_date = '2017-01-01'
test_end_date = '2022-12-31'
rebal_period = '1M'
train_data, test_data = load_feature_data(train_start_date, test_start_date, test_end_date, rebal_period)

/var/folders/1s/hjk6vgwx413bb0gvk7wfl_z40000gn/T/ipykernel_36309/2539331249.py:38: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  input.index = pd.to_datetime(input.index)
/var/folders/1s/hjk6vgwx413bb0gvk7wfl_z40000gn/T/ipykernel_36309/2539331249.py:38: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  input.index = pd.to_datetime(input.index)
/var/folders/1s/hjk6vgwx413bb0gvk7wfl_z40000gn/T/ipykernel_36309/2539331249.py:38: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  input.index = pd.to_datetime(input.index)
/var/folders/1s/hjk6vgwx413bb0gvk7wfl_z40000gn/T/

In [6]:
train_data

,CBOE DJIA Volatility Index,CBOE NASDAQ Volatility Index,CBOE S&P100 Volatility Index,CBOE S&P500 Volatility Index,OptionMetrics - DJI Historical Volatility,OptionMetrics - Russell 2000 Historical Volatility,OptionMetrics - SPX Historical Volatility,OptionMetrics - DJX Index Dividend Yield,OptionMetrics - Russell 2000 Index Dividend Yield,OptionMetrics - SPX Index Dividend Yield
2007-01-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-02-28,-0.167095,-0.022286,-0.139153,-0.143688,0.056294,0.347163,0.000247,0.002343,0.007878,0.000487
2007-03-31,0.543210,0.257160,0.627510,0.534433,0.657119,0.121434,0.655073,0.006376,-0.002570,0.010648
2007-04-30,-0.111333,-0.135286,-0.149291,-0.081542,-0.565452,-0.259023,-0.539878,0.000283,0.011641,-0.001090
2007-05-31,-0.065266,-0.089247,-0.067440,-0.070200,0.153970,0.137907,-0.007005,0.002195,-0.001225,0.002621
...,...,...,...,...,...,...,...,...,...,...
2016-08-31,-0.119215,-0.137255,-0.198561,-0.157752,0.011392,0.092707,-0.015551,-0.003238,-0.001984,0.003680
2016-09-30,0.023089,0.059917,0.108618,0.083601,0.264204,0.276138,0.288959,-0.076863,-0.012372,-0.088924
2016-10-31,0.101167,0.030539,0.090688,0.006677,0.078827,0.123927,0.089788,0.001985,-0.001995,0.002877
2016-11-30,0.293993,0.302648,0.432814,0.367723,-0.344823,-0.243516,-0.313802,0.082508,0.022633,0.105585


In [7]:
test_data

,CBOE DJIA Volatility Index,CBOE NASDAQ Volatility Index,CBOE S&P100 Volatility Index,CBOE S&P500 Volatility Index,OptionMetrics - DJI Historical Volatility,OptionMetrics - Russell 2000 Historical Volatility,OptionMetrics - SPX Historical Volatility,OptionMetrics - DJX Index Dividend Yield,OptionMetrics - Russell 2000 Index Dividend Yield,OptionMetrics - SPX Index Dividend Yield
2017-01-31,-0.039648,-0.065192,-0.160371,-0.086709,-0.576565,-0.210735,-0.553767,0.077485,-0.229253,-0.088487
2017-02-28,-0.087920,-0.184932,-0.080645,-0.080934,0.325206,0.060919,0.312990,0.002849,0.021693,0.010558
2017-03-31,0.040235,-0.014515,0.078486,0.061812,-0.324793,-0.259909,-0.027269,-0.002192,-0.080423,-0.050001
2017-04-30,-0.066076,-0.041085,-0.105308,-0.012759,-0.045414,0.288795,-0.022791,0.000266,-0.007349,-0.001533
2017-05-31,-0.110440,-0.013743,-0.126316,-0.183360,0.173053,-0.067152,-0.158613,-0.004869,-0.007157,0.004462
...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.015541,-0.161074,0.000000,-0.144569,-0.000134,0.008259,0.006775,-0.058136,-0.053839,-0.024893
2022-09-30,0.173615,0.122087,0.000000,0.119089,-0.501981,-0.274523,-0.192629,-0.004205,-0.000011,-0.001428
2022-10-31,0.167716,0.102294,0.000000,0.177621,0.102379,-0.041108,-0.235822,0.064471,0.053731,0.026348
2022-11-30,-0.065075,-0.100394,0.000000,-0.142525,0.821553,0.426008,0.610065,0.007787,-0.000717,0.000873


In [8]:
# get target index data
def index_cum_data_loading(start_date, end_date, rebal_period):
    index_data_dir = os.path.join(os.getcwd(), 'data', 'index_data')
    result_df = pd.DataFrame()
    
    index_data_name_file = {
        'msci_world':'MSCI World.xlsx'
    }
    
    for ind_name in index_data_name_file.keys():
        #load_index_data(self.start_date, self.end_date, index_data_dir, index_data_name_file[ind_name], ind_name)
        input = pd.read_excel(os.path.join(index_data_dir, index_data_name_file[ind_name]), index_col='Date')
        input = input.sort_index(ascending=False)
        reb_index = input.iloc[:, 0].resample(rebal_period).first()
        #calculate percentage change on month level
        index_df = reb_index / reb_index.iloc[0]
        index_df = index_df.fillna(1)
        index_df.rename(ind_name, inplace=True)
        index_df = index_df.loc[start_date:end_date]
        result_df = pd.concat([result_df, index_df], axis=1).fillna(0)
        
    result_df.index = pd.to_datetime(result_df.index)
    return result_df

In [9]:
def plot_target_index(word_index):
    ax = word_index.plot()
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:}".format(float(x))))
    plt.xlabel('Date')
    plt.ylabel('Cumulative P&L')
    plt.title('Cumulative Profit and Loss')
    plt.show()

In [10]:
# loading the target index
word_index = index_cum_data_loading(start_date, end_date, rebal_period)
# rename the target index
word_index.columns = ['msci_world_cum']
plot_target_index(word_index)

NameError: name 'start_date' is not defined

In [ ]:
market_data = data.join(word_index, how='right')
market_data

In [ ]:
market_data.corr()